In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import japanize_matplotlib
from pathlib import Path

# 現在のノートブックからの相対パスを絶対パスに変換
notebook_dir = Path(os.path.abspath('')).parent
data_path = notebook_dir / 'data' / 'processed' / 'wt262avs.adicht_rec2.parquet'

# データの読み込み
df = pd.read_parquet(data_path)

# データの先頭を表示
print("DataFrameの先頭5行:")
display(df.head())

# データの基本情報を表示
print("\nDataFrameの基本情報:")
display(df.info())

DataFrameの先頭5行:


,PFC,PPC,A1,V1,Stimulus,Time_s,PFC_filtered,PPC_filtered,A1_filtered,V1_filtered,Event_ID,Stimulus_Type
0,63.68750,-48.06250,-76.906250,-55.328125,-0.000312,0.000,-9.687126,-29.744644,-29.668319,-31.102712,0,No_Stimulus
1,42.40625,-50.87500,-80.546875,-57.750000,-0.000312,0.001,-16.177164,-31.208505,-31.960842,-31.552850,0,No_Stimulus
2,31.43750,-70.71875,-99.937500,-82.546875,-0.000312,0.002,-22.393838,-32.559073,-34.101970,-31.901094,0,No_Stimulus
3,30.37500,-44.06250,-70.140625,-49.937500,-0.000312,0.003,-28.047092,-33.677845,-35.946312,-32.045870,0,No_Stimulus
4,-4.53125,-58.40625,-88.937500,-58.468750,0.000000,0.004,-32.896286,-34.466580,-37.366260,-31.903339,0,No_Stimulus



DataFrameの基本情報:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4240450 entries, 0 to 4240449
Data columns (total 12 columns):
 #   Column         Dtype  
---  ------         -----  
 0   PFC            float32
 1   PPC            float32
 2   A1             float32
 3   V1             float32
 4   Stimulus       float32
 5   Time_s         float64
 6   PFC_filtered   float64
 7   PPC_filtered   float64
 8   A1_filtered    float64
 9   V1_filtered    float64
 10  Event_ID       int64  
 11  Stimulus_Type  object 
dtypes: float32(5), float64(5), int64(1), object(1)
memory usage: 307.3+ MB


None

In [10]:
# Stimulus_TypeごとのEvent_IDのユニークな数を計算
unique_event_counts = df.groupby('Stimulus_Type')['Event_ID'].nunique()

# 結果を表示
print("Stimulus_TypeごとのEvent_IDのユニークな数:")
display(unique_event_counts)

Stimulus_TypeごとのEvent_IDのユニークな数:


Stimulus_Type
Light          600
Light+Sound    600
No_Stimulus      2
Sound          600
Name: Event_ID, dtype: int64